In [81]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from mlflow.data.pandas_dataset import PandasDataset
from sklearn.base import ClassifierMixin
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, f1_score, recall_score
from sklearn.linear_model import LogisticRegression, RidgeClassifier, ElasticNet
from tqdm import tqdm
import mlflow
from mlflow.models import infer_signature
from catboost import Pool, CatBoostClassifier

In [82]:
processed = pd.read_pickle("data/processed_df.pkl")
X = processed.drop(columns=["Credit_Score"])
y = processed["Credit_Score"]
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33, random_state=42, stratify=y
)
std_scaler = StandardScaler().set_output(transform="pandas")
X_train = std_scaler.fit_transform(X_train)
X_test = std_scaler.transform(X_test)

In [83]:
class CustomCatboostClassifier(CatBoostClassifier, mlflow.pyfunc.PythonModel):
    def predict(data,
        prediction_type='Class',
        ntree_start=0,
        ntree_end=0,
        thread_count=-1,
        verbose=None):
        result = super(CatBoostClassifier).predict(data,
            prediction_type='Class',
            ntree_start=0,
            ntree_end=0,
            thread_count=-1,
            verbose=None)
        if isinstance(result, np.ndarray):
            return result.ravel()

In [84]:
def run_catboost_experiment(experiment_name, catboost_params, suffix=None):
    try:
        experiment_id = mlflow.create_experiment(experiment_name)
    except mlflow.exceptions.MlflowException:
        experiment_id = mlflow.get_experiment_by_name(experiment_name).experiment_id
    
    if suffix is None:
        run_n = "CatBoost"
    else:
        run_n = "CatBoost" + suffix

    with mlflow.start_run(experiment_id=experiment_id, run_name=run_n):
        # mlflow.log_params(catboost_params)
        
        # Prepare the dataset for CatBoost
        train_pool = Pool(X_train, y_train)
        test_pool = Pool(X_test, y_test)
        base_params = {
            'logging_level':'Silent',
            'loss_function':'MultiClass',
            'l2_leaf_reg': 3,
            'random_strength': 1.2,
        }
        cat_param = CatBoostClassifier(**base_params)
        search = cat_param.grid_search(catboost_params, train_pool, cv=5)
        cat_param.fit(train_pool, use_best_model=True, eval_set=test_pool)

        # Extract the best model parameters (for logging)
        best_params = search['params']
        mlflow.log_params((best_params | base_params))

        # Train the best model on the full training set
        model = CatBoostClassifier(**(best_params | base_params))
        model.fit(train_pool, verbose=False)
        
        # Model evaluation
        # y_pred = model.predict(X_test).ravel()
        # print(y_pred)
        # print(type(y_pred))
        # print(y_pred.shape)
        # print(pd.Series(y_pred, name="pred"))

        # log the model into a mlflow ru
    
        # accuracy = accuracy_score(y_test, y_pred)
        # precision = precision_score(y_test, y_pred, average='weighted')
        # recall = recall_score(y_test, y_pred, average='weighted')
        # f1 = f1_score(y_test, y_pred, average='weighted')
        
        # Log model and evaluation metrics
        # signature = infer_signature(X_test, y_test)
        
        mlflow.evaluate(
            lambda x: model.predict(x).ravel(),
            pd.concat([X_test, y_test], axis=1),
            targets="Credit_Score",
            model_type="classifier",
        )
        


In [85]:
run_catboost_experiment("GridSearchCV_", {
    'depth': [2, 3, 4],
    'learning_rate': [0.02, 0.03, 0.04],
    'iterations' : [100, 300, 500, 800],
    })

0:	loss: 0.7415884	best: 0.7415884 (0)	total: 610ms	remaining: 21.3s
1:	loss: 0.7137650	best: 0.7137650 (1)	total: 1.19s	remaining: 20.2s
2:	loss: 0.6980854	best: 0.6980854 (2)	total: 1.81s	remaining: 20s
3:	loss: 0.6830882	best: 0.6830882 (3)	total: 3.58s	remaining: 28.6s
4:	loss: 0.6738794	best: 0.6738794 (4)	total: 5.4s	remaining: 33.5s
5:	loss: 0.6673806	best: 0.6673806 (5)	total: 7.13s	remaining: 35.7s
6:	loss: 0.6710913	best: 0.6673806 (5)	total: 9.76s	remaining: 40.4s
7:	loss: 0.6630521	best: 0.6630521 (7)	total: 12.4s	remaining: 43.5s
8:	loss: 0.6570961	best: 0.6570961 (8)	total: 15.1s	remaining: 45.3s
9:	loss: 0.6609704	best: 0.6570961 (8)	total: 19.4s	remaining: 50.5s
10:	loss: 0.6537773	best: 0.6537773 (10)	total: 23.6s	remaining: 53.7s
11:	loss: 0.6486150	best: 0.6486150 (11)	total: 27.9s	remaining: 55.8s
12:	loss: 0.7183731	best: 0.6486150 (11)	total: 28.5s	remaining: 50.5s
13:	loss: 0.6898757	best: 0.6486150 (11)	total: 29.2s	remaining: 45.9s
14:	loss: 0.6768585	best: 0.6

2024/08/29 17:16:13 INFO mlflow.models.evaluation.default_evaluator: Computing model predictions.
2024/08/29 17:16:13 INFO mlflow.models.evaluation.default_evaluator: The evaluation dataset is inferred as multiclass dataset, number of classes is inferred as 3. If this is incorrect, please specify the `label_list` parameter in `evaluator_config`.
2024/08/29 17:16:13 INFO mlflow.models.evaluation.default_evaluator: Testing metrics on first row...
2024/08/29 17:16:13 WARNING mlflow.models.evaluation.default_evaluator: Skip logging model explainability insights because it requires all label values to be numeric or boolean.
2024/08/29 17:16:13 INFO mlflow.tracking._tracking_service.client: 🏃 View run CatBoost at: http://0.0.0.0:5000/#/experiments/21/runs/1017280dbfa947f18a827534e1c0fbbe.
2024/08/29 17:16:13 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://0.0.0.0:5000/#/experiments/21.
